In [1]:
from scripts import network, differentialEvolution, objFunction
import numpy as np
import pandas as pd
import datetime

### Check objective function: supply-demand ratio

In [22]:
net = network.WaterDistributionNetwork('ctown_pd.inp')
net.set_time_params(duration=3600, hydraulic_step=300)

In [23]:
net.run()

In [24]:
df_links = net.df_links_report
df_nodes = net.df_nodes_report

In [25]:
df_links

link            pumps                                                      \
id                PU1                          PU2                    PU3   
properties       flow     energy status       flow     energy status flow   
00:00:00    96.590064  43.065440    1.0  96.609061  43.060132    1.0  0.0   
00:05:00    96.599741  43.062737    1.0  96.618741  43.057425    1.0  0.0   
00:10:00    96.609324  43.060059    1.0  96.628326  43.054743    1.0  0.0   
00:15:00    96.618863  43.057391    1.0  96.637868  43.052071    1.0  0.0   
00:20:00    96.628358  43.054734    1.0  96.647365  43.049410    1.0  0.0   
00:25:00    96.637805  43.052089    1.0  96.656815  43.046761    1.0  0.0   
00:30:00    96.647198  43.049457    1.0  96.666211  43.044125    1.0  0.0   
00:35:00    96.656543  43.046837    1.0  96.675559  43.041502    1.0  0.0   
00:40:00    96.665841  43.044229    1.0  96.684860  43.038889    1.0  0.0   
00:45:00    96.675096  43.041632    1.0  96.694117  43.036288    1.0  0.0   
00:50:00    96.684299  43.039047    1.0  96.703323  43.033700    1.0  0.0   
00:55:00    96.693455  43.036474    1.0  96.712482  43.031123    1.0  0.0   
00:58:33    96.699920  43.034657    1.0  96.718949  43.029303    1.0  0.0   
01:00:00    96.139140  43.189559    1.0  96.158004  43.184439    1.0  0.0   

link                                 ... valves                             \
id                              PU4  ...     v1       V45                    
properties energy status       flow  ... status  velocity      flow status   
00:00:00      0.0    0.0  33.925675  ...    1.0  0.132768  2.421893    1.0   
00:05:00      0.0    0.0  33.915281  ...    1.0  0.132767  2.421875    1.0   
00:10:00      0.0    0.0  33.904866  ...    1.0  0.132767  2.421875    1.0   
00:15:00      0.0    0.0  33.894426  ...    1.0  0.132767  2.421875    1.0   
00:20:00      0.0    0.0  33.883961  ...    1.0  0.132767  2.421875    1.0   
00:25:00      0.0    0.0  33.873472  ...    1.0  0.132767  2.421875    1.0   
00:30:00      0.0    0.0  33.862959  ...    1.0  0.132767  2.421875    1.0   
00:35:00      0.0    0.0  33.852422  ...    1.0  0.132767  2.421875    1.0   
00:40:00      0.0    0.0  33.841860  ...    1.0  0.132767  2.421875    1.0   
00:45:00      0.0    0.0  33.831275  ...    1.0  0.132767  2.421875    1.0   
00:50:00      0.0    0.0  33.820666  ...    1.0  0.132767  2.421875    1.0   
00:55:00      0.0    0.0  33.810032  ...    1.0  0.132767  2.421875    1.0   
00:58:33      0.0    0.0  33.802466  ...    1.0  0.132767  2.421875    1.0   
01:00:00      0.0    0.0  33.354401  ...    1.0  0.097458  1.777796    1.0   

link                                                                
id               V47                         V2                     
properties  velocity      flow status  velocity        flow status  
00:00:00    0.281040  2.278495    1.0  2.063351  104.552098    1.0  
00:05:00    0.281038  2.278476    1.0  2.060275  104.396247    1.0  
00:10:00    0.281038  2.278476    1.0  2.057212  104.241037    1.0  
00:15:00    0.281038  2.278476    1.0  2.054161  104.086449    1.0  
00:20:00    0.281038  2.278476    1.0  2.051123  103.932489    1.0  
00:25:00    0.281038  2.278476    1.0  2.048097  103.779161    1.0  
00:30:00    0.281038  2.278476    1.0  2.045083  103.626465    1.0  
00:35:00    0.281038  2.278476    1.0  2.042082  103.474390    1.0  
00:40:00    0.281038  2.278476    1.0  2.039094  103.322953    1.0  
00:45:00    0.281038  2.278476    1.0  2.036117  103.172133    1.0  
00:50:00    0.281038  2.278476    1.0  2.033153  103.021944    1.0  
00:55:00    0.281038  2.278476    1.0  2.030201  102.872379    1.0  
00:58:33    0.281038  2.278476    1.0  2.028114  102.766594    1.0  
01:00:00    0.206299  1.672544    1.0  2.057531  104.257180    1.0  

[14 rows x 45 columns]

In [27]:
objFunction.supply_demand_ratio(net, ['PU1','PU2'])

2137.4297444176477
2705.357330064271


1.2657058493407267

In [28]:
objFunction.supply_demand_ratio2(net, ['PU1','PU2'])

1.265705849340726

In [29]:
total_demand = 0
for junc_id in net.junctions.uid:
    total_demand += df_nodes['junctions', junc_id, 'demand'].sum()
total_demand

2137.4297444176486

New method: sum of the ratios at each timestamp

In [21]:
tot_r = 0
tot_d = 0 
tot_ratio = 0

for time in df_links.index:
    demand = sum([df_nodes.loc[time]['junctions', junc_id, 'demand'] for junc_id in net.junctions.uid])
    flow = sum([df_links.loc[time]['pumps', pump_id, 'flow'] for pump_id in net.pumps.uid])
    ratio = flow/demand
    print(ratio)
    tot_ratio += ratio
tot_ratio

0.47321270587247943
0.4732270868317517
0.9999996777331489
0.9999996782999285
0.9999996770153096
0.9999996778224257
0.9999996777497266
0.999999677803064
0.9999996779088709
0.9999996776148276
0.9999996769747629
0.9999996769589696
0.9999996779063294
0.9999996780084267
0.9999996783057898


13.946435602805812

### Differential Evolution

In [2]:
 DE_params = {
        'vars_list': ['78', '79'],
        #'vars_list': ['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11'],
        'n_pop': 5,
        'n_generations': 1,
        'CR': 0.7,
        'F': 0.5,
    }

In [3]:
net = network.WaterDistributionNetwork("anytown_pd.inp")
de = differentialEvolution.DifferentialEvolution(net, objFunction.supply_demand_ratio, **DE_params)

In [4]:
duration = 86400           # 1 day
hyd_step = 600             # 10 min
update_interval = 14400    # 4 hours

One day simulation

In [ ]:
de.run(duration, hyd_step, update_interval)

One week simulation

In [2]:
 DE_params = {
        'vars_list': ['78', '79'],
        #'vars_list': ['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11'],
        'n_pop': 20,
        'n_generations': 5,
        'CR': 0.7,
        'F': 0.5,
    }

In [3]:
net = network.WaterDistributionNetwork("anytown_pd.inp")
de = differentialEvolution.DifferentialEvolution(net, objFunction.supply_demand_ratio, **DE_params)

In [4]:
pattern_csv = "../demand_patterns/demands_anytown.csv"
junc_demands = pd.read_csv(pattern_csv, names=['junc_demand'])
pattern_dict = {col_name: junc_demands[col_name].values for col_name in junc_demands.columns}
pattern_dict

{'junc_demand': array([0.4323678 , 0.31463272, 0.28294762, 0.31342878, 0.38992098,
        0.53782283, 0.72196949, 0.83094734, 0.89109183, 0.70566872,
        0.67892882, 0.64078848, 0.58551938, 0.48666518, 0.57186794,
        0.53649159, 0.56839894, 0.59404046, 0.62807841, 0.57891847,
        0.50930043, 0.47389309, 0.35215888, 0.58658994, 0.49251589,
        0.34361989, 0.33491218, 0.34594515, 0.4181497 , 0.64427033,
        0.69326868, 0.87916386, 0.68262554, 0.71628208, 0.63443703,
        0.58261758, 0.51512651, 0.48931437, 0.50384703, 0.55066237,
        0.63881162, 0.59381814, 0.58565399, 0.49547184, 0.53097953,
        0.47264104, 0.4739089 , 0.58101241, 0.45732562, 0.34488161,
        0.34429857, 0.33160492, 0.43458401, 0.52734943, 0.6887121 ,
        0.77304824, 0.82584896, 0.76018191, 0.66211617, 0.62916935,
        0.5834599 , 0.49584871, 0.54656121, 0.55780108, 0.56629535,
        0.54395235, 0.55287307, 0.56756655, 0.60097603, 0.46478011,
        0.38291637, 0.58100747, 0

In [5]:
net.set_demand_pattern('junc_demand', junc_demands.values, net.junctions)

In [6]:
days = 7

duration = 24*3600          # 7 days
hyd_step = 600              # 10 min
update_interval = 3600*4    # 4 hours

In [7]:
de.run(duration, hyd_step, update_interval, days, keep_same_pop=False, save_results=True, info='week_keep_pop_complex')

DAY 1:
> STARTING GENERATION 1: ...
>>> Parent: candidate = {'78': array([0, 1, 0, 1, 0, 0]), '79': array([1, 0, 1, 0, 1, 0])}, value = 0.9945817505519448
>>> Child: candidate = {'78': array([1, 1, 1, 0, 0, 0]), '79': array([1, 0, 0, 1, 1, 0])}, value = 0.9899093987286701
    -> NEW BEST CANDIDATE FOUND!
    --> Substitution: False
 ----- Elapsed time [0:00:06.177597] -----
>>> Parent: candidate = {'78': array([1, 0, 0, 0, 1, 1]), '79': array([1, 1, 1, 0, 0, 1])}, value = 0.9884297589576282
>>> Child: candidate = {'78': array([1, 0, 0, 0, 1, 0]), '79': array([1, 1, 1, 0, 0, 1])}, value = 0.9853485363111137
    --> Substitution: False
 ----- Elapsed time [0:00:12.327835] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 1, 0, 0, 0, 0])}, value = 0.9947188045579998
>>> Child: candidate = {'78': array([0, 0, 1, 0, 1, 0]), '79': array([0, 0, 0, 0, 0, 1])}, value = 0.981325297245208
    -> NEW BEST CANDIDATE FOUND!
    --> Substitution: False
 ----- Elapsed tim

>>> Child: candidate = {'78': array([0, 1, 1, 0, 0, 1]), '79': array([0, 1, 0, 0, 1, 0])}, value = 0.9858032220216751
    --> Substitution: False
 ----- Elapsed time [0:02:33.362435] -----
>>> Parent: candidate = {'78': array([1, 1, 0, 1, 0, 0]), '79': array([0, 0, 1, 1, 1, 1])}, value = 0.9917249430994108
>>> Child: candidate = {'78': array([0, 1, 1, 1, 0, 0]), '79': array([0, 1, 1, 0, 1, 0])}, value = 0.9913552592654302
    --> Substitution: False
 ----- Elapsed time [0:02:39.102455] -----
>>> Parent: candidate = {'78': array([0, 1, 1, 0, 1, 0]), '79': array([1, 0, 0, 0, 1, 0])}, value = 0.9933550007559482
>>> Child: candidate = {'78': array([0, 1, 1, 0, 0, 0]), '79': array([0, 1, 0, 1, 0, 1])}, value = 0.9856763458582377
    --> Substitution: False
 ----- Elapsed time [0:02:44.846969] -----
>>> Parent: candidate = {'78': array([1, 0, 1, 1, 1, 0]), '79': array([0, 1, 0, 0, 1, 1])}, value = 0.9917476810291793
>>> Child: candidate = {'78': array([0, 1, 0, 1, 1, 1]), '79': array([0, 1, 

>>> Child: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 0, 0, 0, 0, 0])}, value = 0.9874147156277051
    --> Substitution: False
 ----- Elapsed time [0:06:36.673922] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 1, 1, 0]), '79': array([0, 0, 0, 0, 0, 0])}, value = 0.9945817497535371
>>> Child: candidate = {'78': array([0, 0, 0, 1, 1, 1]), '79': array([0, 0, 0, 0, 0, 0])}, value = 0.9806807404194423
    --> Substitution: False
 ----- Elapsed time [0:06:46.879018] -----
>>> Parent: candidate = {'78': array([1, 0, 1, 0, 1, 0]), '79': array([0, 1, 1, 0, 1, 0])}, value = 0.9935248227520533
>>> Child: candidate = {'78': array([1, 1, 1, 1, 1, 0]), '79': array([0, 0, 0, 1, 1, 1])}, value = 0.9918296072455409
    --> Substitution: False
 ----- Elapsed time [0:06:55.924367] -----
>>> Parent: candidate = {'78': array([0, 1, 0, 1, 1, 0]), '79': array([1, 0, 1, 1, 0, 1])}, value = 0.9917249438782242
>>> Child: candidate = {'78': array([0, 1, 0, 1, 1, 1]), '79': array([0, 0, 

>>> Child: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 0, 1, 1, 0, 1])}, value = 0.9879818123058227
    --> Substitution: False
 ----- Elapsed time [0:09:42.261771] -----
>>> Parent: candidate = {'78': array([0, 1, 0, 1, 1, 1]), '79': array([1, 0, 0, 0, 0, 1])}, value = 0.992894666562048
>>> Child: candidate = {'78': array([1, 1, 0, 0, 0, 0]), '79': array([1, 0, 0, 1, 0, 1])}, value = 0.9810563668182086
    --> Substitution: False
 ----- Elapsed time [0:09:49.100396] -----
>>> Parent: candidate = {'78': array([0, 1, 0, 1, 1, 1]), '79': array([1, 1, 1, 0, 0, 1])}, value = 0.993082132387064
>>> Child: candidate = {'78': array([0, 1, 0, 1, 0, 1]), '79': array([1, 0, 0, 0, 0, 1])}, value = 0.9910022380540104
    --> Substitution: False
 ----- Elapsed time [0:09:56.375432] -----
> GENERATION DONE!
> ------------------------------------------------
> Best generation member: {'78': array([0, 1, 1, 1, 1, 0]), '79': array([1, 0, 1, 1, 1, 0])}
> Best generation value: 0.9948354

>>> Parent: candidate = {'78': array([0, 0, 0, 0, 1, 1]), '79': array([0, 1, 0, 0, 1, 0])}, value = 0.974313522386205
>>> Child: candidate = {'78': array([0, 0, 1, 0, 0, 0]), '79': array([0, 1, 0, 1, 0, 1])}, value = 0.9801634849656953
    --> Substitution: True
 ----- Elapsed time [0:12:32.366323] -----
>>> Parent: candidate = {'78': array([0, 0, 1, 1, 1, 1]), '79': array([1, 1, 0, 0, 1, 1])}, value = 0.9917103044307639
>>> Child: candidate = {'78': array([0, 0, 1, 1, 0, 0]), '79': array([1, 0, 1, 0, 1, 0])}, value = 0.9853361221660751
    --> Substitution: False
 ----- Elapsed time [0:12:38.986951] -----
>>> Parent: candidate = {'78': array([1, 1, 0, 1, 0, 0]), '79': array([1, 1, 1, 1, 0, 0])}, value = 0.9895319896407144
>>> Child: candidate = {'78': array([0, 1, 0, 1, 0, 1]), '79': array([1, 1, 1, 0, 0, 1])}, value = 0.9928693570256956
    --> Substitution: True
 ----- Elapsed time [0:12:45.886615] -----
>>> Parent: candidate = {'78': array([1, 1, 0, 0, 1, 0]), '79': array([1, 0, 0,

...

>>> Parent: candidate = {'78': array([1, 1, 1, 1, 1, 0]), '79': array([1, 1, 1, 1, 1, 1])}, value = 0.9977380576893171
>>> Child: candidate = {'78': array([1, 0, 1, 1, 1, 0]), '79': array([1, 1, 1, 1, 0, 0])}, value = 0.9958928169872003
    --> Substitution: False
 ----- Elapsed time [1:01:39.295938] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 0, 0, 0]), '79': array([0, 1, 0, 1, 1, 1])}, value = 0.9946007665194603
>>> Child: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 1, 0, 1, 1, 0])}, value = 0.9972505853211427
    --> Substitution: True
 ----- Elapsed time [1:01:45.062484] -----
>>> Parent: candidate = {'78': array([0, 0, 0, 1, 1, 1]), '79': array([0, 0, 1, 1, 0, 0])}, value = 0.9915343850021472
>>> Child: candidate = {'78': array([0, 1, 0, 1, 1, 0]), '79': array([1, 0, 1, 1, 1, 0])}, value = 0.9913893265374222
    --> Substitution: False
 ----- Elapsed time [1:01:50.625375] -----
>>> Parent: candidate = {'78': array([1, 0, 1, 1, 1, 1]), '79': array([1, 1, 

>>> Child: candidate = {'78': array([0, 1, 0, 0, 1, 0]), '79': array([0, 1, 1, 1, 1, 1])}, value = 0.9929509396815612
    --> Substitution: True
 ----- Elapsed time [1:04:01.943020] -----
>>> Parent: candidate = {'78': array([0, 1, 0, 0, 1, 1]), '79': array([1, 0, 1, 0, 1, 0])}, value = 0.9905333541103997
>>> Child: candidate = {'78': array([0, 1, 1, 0, 1, 0]), '79': array([1, 1, 1, 0, 0, 1])}, value = 0.9913340052295948
    --> Substitution: True
 ----- Elapsed time [1:04:07.724179] -----
>>> Parent: candidate = {'78': array([1, 1, 0, 0, 1, 0]), '79': array([1, 0, 0, 1, 1, 1])}, value = 0.9948308230922153
>>> Child: candidate = {'78': array([1, 1, 1, 1, 1, 0]), '79': array([0, 1, 0, 1, 1, 1])}, value = 0.9927065609892389
    --> Substitution: False
 ----- Elapsed time [1:04:13.464180] -----
>>> Parent: candidate = {'78': array([0, 1, 1, 0, 0, 0]), '79': array([0, 1, 1, 0, 0, 1])}, value = 0.9888492581911693
>>> Child: candidate = {'78': array([0, 1, 0, 0, 0, 0]), '79': array([0, 0, 1,

>>> Child: candidate = {'78': array([0, 1, 0, 0, 1, 1]), '79': array([0, 1, 1, 1, 1, 1])}, value = 0.9912096327620343
    --> Substitution: True
 ----- Elapsed time [1:06:31.183387] -----
> GENERATION DONE!
> ------------------------------------------------
> Best generation member: {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 1, 0, 0, 1, 1])}
> Best generation value: 0.9964730739396009
> ------------------------------------------------
> STARTING GENERATION 3: ...
>>> Parent: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 1, 0, 0, 1, 1])}, value = 0.9964730739396009
>>> Child: candidate = {'78': array([1, 1, 0, 0, 1, 0]), '79': array([1, 1, 0, 0, 0, 1])}, value = 0.9954874310643446
    --> Substitution: False
 ----- Elapsed time [1:06:36.989408] -----
>>> Parent: candidate = {'78': array([0, 1, 1, 0, 0, 0]), '79': array([0, 1, 1, 1, 0, 1])}, value = 0.9913456666680082
>>> Child: candidate = {'78': array([1, 1, 1, 0, 1, 0]), '79': array([1, 1, 0, 1, 0, 0])}, value =

>>> Child: candidate = {'78': array([0, 0, 1, 0, 1, 1]), '79': array([0, 0, 1, 0, 1, 0])}, value = 0.9886366006185522
    --> Substitution: False
 ----- Elapsed time [1:08:55.233651] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 1, 1, 1]), '79': array([0, 1, 1, 1, 1, 0])}, value = 0.9928129673254849
>>> Child: candidate = {'78': array([0, 1, 0, 1, 0, 1]), '79': array([1, 1, 1, 1, 1, 0])}, value = 0.9926293152595776
    --> Substitution: False
 ----- Elapsed time [1:09:00.830185] -----
>>> Parent: candidate = {'78': array([0, 0, 1, 1, 0, 1]), '79': array([0, 1, 0, 1, 1, 1])}, value = 0.9910375983557544
>>> Child: candidate = {'78': array([1, 0, 0, 0, 1, 0]), '79': array([1, 0, 0, 0, 0, 0])}, value = 0.9573670094374244
    --> Substitution: False
 ----- Elapsed time [1:09:06.611312] -----
>>> Parent: candidate = {'78': array([1, 0, 0, 0, 0, 1]), '79': array([0, 1, 1, 1, 0, 0])}, value = 0.9903276713924223
>>> Child: candidate = {'78': array([1, 0, 1, 0, 0, 1]), '79': array([0, 1, 

>>> Parent: candidate = {'78': array([1, 1, 0, 0, 1, 0]), '79': array([1, 1, 1, 1, 0, 0])}, value = 0.9963990257127191
>>> Child: candidate = {'78': array([0, 1, 0, 0, 1, 1]), '79': array([0, 1, 1, 1, 0, 0])}, value = 0.9928632317844184
    --> Substitution: False
 ----- Elapsed time [1:11:23.866081] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 1, 1, 0]), '79': array([1, 1, 1, 0, 1, 1])}, value = 0.9970678181625431
>>> Child: candidate = {'78': array([1, 1, 0, 0, 1, 0]), '79': array([1, 1, 1, 0, 1, 0])}, value = 0.9956280279752496
    --> Substitution: False
 ----- Elapsed time [1:11:29.632607] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 0, 0, 1]), '79': array([0, 0, 1, 0, 1, 1])}, value = 0.9888335867183555
>>> Child: candidate = {'78': array([1, 1, 1, 0, 0, 0]), '79': array([1, 1, 0, 1, 1, 0])}, value = 0.996399026121031
    --> Substitution: True
 ----- Elapsed time [1:11:35.428140] -----
>>> Parent: candidate = {'78': array([1, 1, 1, 0, 1, 1]), '79': array([1, 0, 1